In [31]:
%config IPCompleter.greedy=True
from functools import reduce
from itertools import accumulate
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from scipy import linalg
from scipy.stats import mstats
from scipy.optimize import minimize
from scipy.optimize import brentq
import matplotlib.pyplot as plt
import seaborn as sns
import pdb
import copy
import math
from sklearn import datasets as ds
from sklearn import model_selection
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
%matplotlib inline

In [12]:
#!pip install pandas
#!pip install plotly
#!pip install scipy
#!pip install matplotlib
#!pip install seaborn
#!pip install sklearn

    100% |████████████████████████████████| 12.4MB 2.2MB/s ta 0:00:011
  Running setup.py bdist_wheel for sklearn ... done
  Stored in directory: /home/slava/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [54]:
plotly.tools.set_credentials_file(username='slava-mazur', api_key='0Up94ov3B6l4c11Pbxrc')

In [3]:
my_seed = 777
test_size = 0.5

In [4]:
from distutils.sysconfig import get_python_lib
print(get_python_lib())

/home/ilongin/anaconda3/lib/python3.6/site-packages


In [5]:
X, y = ds.load_iris(return_X_y=True)
X[0:5,:]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [6]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = test_size, random_state = my_seed)
X_train.shape

(75, 4)

In [7]:
# Create a plotly trace
def plotly_scatter(data, Class = None, text = None, show_text = False, marker_size = 3, 
                   colors = ['blue', 'red', 'gray'], fname = 'my-scatter'):
    if (text is None) or not show_text:
        mode = 'markers'  
    else: 
        mode = 'markers+text'
    traces = list()
    if Class is None:
        if data.shape[1] > 2:
            trace = go.Scatter3d(
                x = data[:,0],
                y = data[:,1],
                z = data[:,2],
                mode = mode,
                marker = dict(size = marker_size, color = colors[0], autocolorscale = False, 
                              showscale = False),
                showlegend = False,
                text = text,
                hovertext = text,
                hoverinfo = "text"
            )
        else:
            trace = go.Scatter(
                x = data[:,0],
                y = data[:,1],
                mode = mode,
                marker = dict(size = marker_size, color = colors[0], autocolorscale = False, 
                              showscale = False),
                showlegend = False,
                text = text,
                hovertext = text,
                hoverinfo = "text"
            )
            
        traces.append(trace)
    else:
        classes = set(Class)
        clr = 0
        for c in classes:
            if text is None:
                txt = None
                hoverinfo = "all"
            else:
                txt = np.array(text)[Class == c]
                hoverinfo = "text"
            if data.shape[1] > 2:
                trace = go.Scatter3d(
                    name = c,
                    x = data[Class == c, 0],
                    y = data[Class == c, 1],
                    z = data[Class == c, 2],
                    mode = mode,
                    marker = dict(size = marker_size, color = colors[clr], autocolorscale = False, 
                              showscale = False),
                    showlegend = True,
                    text = txt,
                    hovertext = txt,
                    hoverinfo = hoverinfo
                )
            else:
                trace = go.Scatter(
                    name = c,
                    x = data[Class == c, 0],
                    y = data[Class == c, 1],
                    mode = mode,
                    marker = dict(size = marker_size, color = colors[clr], autocolorscale = False, 
                              showscale = False),
                    showlegend = True,
                    text = txt,
                    hovertext = txt,
                    hoverinfo = hoverinfo
                )
                
            traces.append(trace)
            clr += 1
            
    
    layout = go.Layout(showlegend=(Class is not None))
    fig = go.Figure(data=traces, layout=layout)

    return py.iplot(fig, filename=fname)

In [20]:
# some variety of kernel functions
def gauss_kernel_norm(x):
    vars_ = [np.var(x[:, j]) for j in range(x.shape[1])]
    
    def calc(y, z):
        tmp = y - z
        return np.exp(-np.mean(tmp * tmp / vars_))
    
    return calc

def mahalanobis_kernel_norm(x):
    cov_ =  np.linalg.inv(np.cov(x, rowvar=False))
    
    def calc(y, z):
        tmp = y - z
        d = np.dot(np.dot(tmp.T, cov_), tmp)
        return np.exp(-d / len(y))
    
    return calc

def dot_kernel_norm(y, z):
    cv = np.inner(y, z)
    ny = np.inner(y, y)
    nz = np.inner(z, z)
    return cv / (np.sqrt(ny) * np.sqrt(nz))

def dot_kernel(y, z):
    cv = np.inner(y, z)
    return cv / len(y)



In [19]:
# Kernel (Gram) matrix calculation and centering
def kern_matrix(data, fKernel):
    K = np.zeros([data.shape[0], data.shape[0]]) 
    for i in range(data.shape[0]): 
        for j in range(i, data.shape[0]):
            K[i, j] = K[j, i] = fKernel(data[i,:], data[j, :])
    return K

def kern_matrix_test(X, Y, fKernel):
    K = np.zeros([X.shape[0], Y.shape[0]]) 
    for i in range(X.shape[0]): 
        for j in range(Y.shape[0]):
            K[i, j] = fKernel(X[i,:], Y[j, :])
    return K

def kern_matrix_centered(data, basis_events, u, fKernel):
    K0 = kern_matrix(basis_events, fKernel)
    sq_norm = np.dot(u.T, np.dot(K0, u))
    K = kern_matrix(data, fKernel)
    Y = np.zeros([data.shape[0], basis_events.shape[0]])
    for i in range(data.shape[0]): 
        for j in range(basis_events.shape[0]):
            Y[i, j] = fKernel(data[i,:], basis_events[j, :])
    y = np.dot(Y, np.reshape(u, [basis_events.shape[0], 1]))
    
    for j in range(K.shape[1]): 
        K[:, [j]] -= y
    y = y.T
    for i in range(K.shape[0]): 
        K[[i], :] -= y
    return K + sq_norm

def kern_matrix_centered_test(X, Y, basis_events, u, fKernel):
    K0 = kern_matrix(basis_events, fKernel)
    sq_norm = np.dot(u.T, np.dot(K0, u))
    K = kern_matrix_test(X, Y, fKernel)
    Z = np.zeros([X.shape[0], basis_events.shape[0]])
    for i in range(X.shape[0]): 
        for j in range(basis_events.shape[0]):
            Z[i, j] = fKernel(X[i,:], basis_events[j, :])
    y = np.dot(Z, np.reshape(u, [basis_events.shape[0], 1]))
    
    for j in range(K.shape[1]): 
        K[:, [j]] -= y
        
    Z = np.zeros([Y.shape[0], basis_events.shape[0]])
    for i in range(Y.shape[0]): 
        for j in range(basis_events.shape[0]):
            Z[i, j] = fKernel(Y[i,:], basis_events[j, :])
    y = np.dot(Z, np.reshape(u, [basis_events.shape[0], 1]))
    y = y.T
    for i in range(K.shape[0]): 
        K[[i], :] -= y
        
    return K + sq_norm


In [11]:
# Distance-to-center related calculations
def calc_rmin2(K):
    k = (K - np.identity(K.shape[0])).flatten().argmax()
    i = k // K.shape[0]
    j = k % K.shape[0]
    return K[i, i] + K[j, j] - 2*K[i, j]

# formula for 'b_n' in the paper
def square_dists_to_center(K, w):
    w2 = w * w
    w2 = w2 / np.sum(w2)
    return K.diagonal() - 2 * np.dot(K, w2) + np.dot(np.dot(w2.transpose(), K), w2)

# heuristic initial point
def init_approx2(K, rmin2):
    w = np.maximum(K.sum(axis=0), 0) 
    r2 = square_dists_to_center(K, np.sqrt(w))
    a = np.sqrt(r2.max() * r2.min())
    w = a / (a + r2)
    r = np.sqrt(rmin2 / 2);
    if (r > 0):
        amax = r
        wmax = amax / (amax + r*r)
    else: 
        wmax = 1
    
    return np.minimum(w, wmax-1e-6)

# scaling factor 'a'
def scale_factor(K, w):
    R = square_dists_to_center(K, w)
    return np.sqrt(np.prod([R.min(), R.max()]))

# scaling factor for basis events only (do we need this?)
def scale_factor_reduced(K, w, rmin2):
    R = square_dists_to_center(K, w)
    R = [R.min(), R.max()]
    rmin = [rmin2/2, rmin2/2]
    return np.sqrt(np.prod(np.maximum(R, rmin)))

# function 'J' to be minimized by 'w'
def func_j2(w, K, debug_print = False):
    #pdb.set_trace()
    a = scale_factor(K, w)
    w2 = w * w
    sumw2 = np.sum(w2)
    res = 0.5 * (np.sum(K.diagonal() * w2) - np.dot(np.dot(w2.transpose(), K), w2) / sumw2 + a * np.sum((1 - w)**2)) / K.shape[0]
    if debug_print:
        print("a = {}, wmin = {}, wmax = {}, value = {}".format(a, w.min(), w.max(), res))
    return res.flatten()[0]

# pseudo-gradient of 'J' (treats scaling factor to be independent of 'w')
def func_j2_pseudo_grad(w, K):
    a = scale_factor(K, w)
    w2 = w * w
    sumw2 = np.sum(w2)
    Kd = K.diagonal().flatten()
    Kw2 = np.dot(w2.transpose(), K).flatten()
    res = Kd * w - w * (Kw2 + w2 * (Kd - Kw2)) / (sumw2 * sumw2) - a * (1 - w) 
    return res / K.shape[0]



In [18]:
def calc_pseudo_opt_weights(K, method = 'COBYLA'):
    rmin2 = calc_rmin2(K)
    w = init_approx2(K, rmin2)
    print("Minimization of J using {}: function value at the initial point: {}".format(method, func_j2(w, K)))
    non_grad = set(['Nelder-Mead', 'COBYLA'])
    if method in non_grad:
        o = minimize(fun = func_j2, x0 = w, args = (K), method = method, options={'disp':False})
    else:
        o = minimize(fun = func_j2, x0 = w, args = (K), method = method, jac=func_j2_pseudo_grad, options={'disp':True})
    print("Done: function value at the final point: {}".format(o['fun']))
    return o['x']

In [32]:
# Sugeno measure and integral related calculations
def SugenoMeasure(w, lmbda):
    return reduce(lambda w, R: w + R + lmbda * w * R, w)

def SugenoMeasureCum(w, lmbda):
    res = np.zeros(len(w), dtype = float)
    for i, x in enumerate(accumulate(w, lambda w, R: w + R + lmbda * w * R)):
        res[i] = x
    return res

Frank0 = lambda y, Fn, K, X: sum(np.maximum(Fn[X], K[y, X])) / len(X)

def SugenoInt(K, Frank, Fn, w, X, lmbda = None):
    bx = np.array([Frank(y, Fn, K, X) for y in X], dtype=np.float)
    ibx = (-bx).argsort()
    jbx = X[ibx]
    bx = bx[ibx]
    w = w[jbx]
    if (lmbda is None) and (len(w) > 1):
        f = lambda x: SugenoMeasure(w, x) - 1
        
        m = SugenoMeasure(w, 0)
        if (m > 1):
            lmbda = brentq(f, -1, 0, maxiter=100, full_output=False, disp=False)
        elif (m < 1):
            if (m < 1e-6): # measure of X is practically zero -- need to stop 
                return []
            
            rb = 1
            while m < 1:
                m = SugenoMeasure(w, rb)
                if m < 1:
                    rb = 2 * rb 
            lmbda = brentq(f, 0, rb, maxiter=100, full_output=False, disp=False)
        else: 
            lmbda = 0
            
        m = SugenoMeasureCum(w, lmbda)
    else:
        if (lmbda is None): 
            lmbda = 0
        m = SugenoMeasureCum(w, lmbda)
        m = m / m[-1]
    
    res = np.minimum(bx, m)
    return res[np.argsort(ibx)]

def BasisEvents(K, Frank, w, lmbda = None, eps = 1e-4, min_events = 3, control_singularity = False):
    # pdb.set_trace()
    res = []
    N = len(w)
    X = np.array(range(N), dtype=np.integer)
    Fn = np.repeat(0.0, N)
    sMax = 0.0
    w0 = copy.deepcopy(w)
    for i in range(N):
        S = SugenoInt(K, Frank, Fn, w0, X, lmbda)
        if (S is None) or (len(S) == 0): 
            break
        Fn1 = copy.deepcopy(Fn)
        Fn[X] = S; 
        progress = min(max(abs(Fn[X]-Fn1[X])), max(S) - sMax)
        jmax = np.argmax(S)
        sMax = S[jmax]
        j = X[jmax]
        res.append(j)
        Xj = set(X) - set([j])
        X = np.sort(list(Xj))
        if (X is None) or (len(X) == 0): 
            break
        Fn[j] = 0.0;
        print("step: {}; selected: {}; sMax = {}; progress = {}".format(i, j, sMax, progress))
        w0[j] = 0.0
        if (control_singularity) and (len(res) > 1):
            Ku = K[res, :][:, res]
            v = linalg.eigh(Ku, eigvals_only = True)
            if (min(v) < 1e-8):
                print("adding event %d leads to a singular matrix, omitting." % j)
                res = res[:-1]
                progress = 1.0
                sMax = 0.0

        if (len(res) >= min_events) and ((progress < eps) or (sMax > 1 - eps)): 
            break
    
    if (progress < 0) and (len(res) > min_events): 
        return res[0:(len(res)-1)] 
    else: 
        return res



In [22]:
# Center related calculations
def BasisEventsWeights(K, be, w):
    b = np.dot(K[be, :], w / np.sum(w))
    return np.linalg.solve(K[be, :][:, be], b).flatten()

def square_dists_to_center_reduced(K, u, be):
    res = K.diagonal() - 2 * np.dot(K[:, be], u) + np.dot(np.dot(u.transpose(), K[be, :][:, be]), u)
    return res.flatten()

def center_kern_matrix(K, u):
    y = np.dot(K, u)
    sqnorm = np.dot(u.transpose(), y)
    for i in range(K.shape[0]):
        K[i, :] -= y
    for j in range(K.shape[1]):
        K[:, j] -= y
    return K + sqnorm


In [23]:
# Non-linear eigenvector decomposition
def nl_eigen_vectors(K, L, normalize = False):
    N = K.shape[0]
    w, v = linalg.eigh(K, eigvals = (N-L, N-1))
    if (normalize):
        for i in range(v.shape[0]):
            v[i, :] /= np.sqrt(w) 
    return v[:,::-1]

In [24]:
# Membership and non-linear PCA related calculations
def predict_memberships(x, basis_events, fKernel, u, a, L = 3, normalize = True):
    # pdb.set_trace()
    res = np.zeros([x.shape[0], L+1])
    K0 = kern_matrix_centered(x, basis_events, u, fKernel)
    res[:,0] = a / (a + K0.diagonal())
    if L > 0:
        U = nl_eigen_vectors(K0, L, normalize=normalize)
        tmp = np.dot(U.T, K0)
        res[:,1:] = tmp.T
    return res 

In [29]:
# putting all together
def append_membership_feature(x_train, y_train, x_test, fKernel = dot_kernel_norm, cluster = 0, L = 3, normalize = True):
    #pdb.set_trace()
    M = x_train.shape[1]
    data = x_train[y_train == cluster, :]
    print("Calculating kernel")
    K = kern_matrix(data, fKernel)
    print("Finding pseudo-optimal center coeffs")
    w = calc_pseudo_opt_weights(K)
    w2 = w * w
    w2 = w2 / np.sum(w2)
    print("Finding principal events")
    be = BasisEvents(K, Frank=Frank0, w=w2, lmbda = None, eps = 1e-4, min_events = L, control_singularity = True) 
    u = BasisEventsWeights(K, be, w2)
    r2 = square_dists_to_center(K, w)
    a = np.sqrt(np.prod([r2.min(), r2.max()]))
    print("Scale: {}, membership range: {} - {}".format(a, a / (a + r2.max()), a / (a + r2.min())))

    basis_events = data[be, :]
    
    print("Calculating memberships")
    y = predict_memberships(x_train, basis_events, fKernel, u, a, L = L, normalize = normalize)
    y1 = predict_memberships(x_test, basis_events, fKernel, u, a, L = L, normalize = normalize)
    res = {'x_train': np.append(x_train, y, axis = 1), 'x_test': np.append(x_test, y1, axis = 1), 'basis_events': basis_events, 'u':u, 'a':a}
    print("Done")
    
    return res

In [33]:
z0 = append_membership_feature(X_train, y_train, X_test, fKernel = dot_kernel_norm, cluster = 0, L = 3, normalize=True)

Calculating kernel
Finding pseudo-optimal center coeffs
Minimization of J using COBYLA: function value at the initial point: 0.00011600072106008173
Done: function value at the final point: 1.674590900914595e-05
Finding principal events
step: 0; selected: 9; sMax = 0.997380960236043; progress = 0.997380960236043
step: 1; selected: 15; sMax = 0.9984260256280928; progress = 0.0010450653920498487
step: 2; selected: 7; sMax = 0.9989278300268044; progress = 0.0005018043987116139
step: 3; selected: 1; sMax = 0.9990400779038777; progress = 0.00011224787707331263
step: 4; selected: 21; sMax = 0.9991354701977657; progress = 9.539229388799697e-05
adding event 21 leads to a singular matrix, omitting.
step: 5; selected: 2; sMax = 0.9991205394774028; progress = 0.6480520992902261
adding event 2 leads to a singular matrix, omitting.
step: 6; selected: 13; sMax = 0.9991232540907848; progress = 0.34359290334072734
adding event 13 leads to a singular matrix, omitting.
step: 7; selected: 4; sMax = 0.9990

In [34]:
z1 = append_membership_feature(X_train, y_train, X_test, fKernel = dot_kernel_norm, cluster = 1, L = 3, normalize=True)

Calculating kernel
Finding pseudo-optimal center coeffs
Minimization of J using COBYLA: function value at the initial point: 0.0001840378541852153
Done: function value at the final point: 2.358981453583084e-05
Finding principal events
step: 0; selected: 31; sMax = 0.9978889129926314; progress = 0.9978889129926314
step: 1; selected: 19; sMax = 0.9982586944172733; progress = 0.0003697814246419462
step: 2; selected: 12; sMax = 0.9984299802730103; progress = 0.00017128585573700228
step: 3; selected: 2; sMax = 0.9984944181426271; progress = 6.443786961674025e-05
Scale: 9.024089278250313e-06, membership range: 0.0010310102932314493 - 0.9989689897067685
Calculating memberships
Done


In [35]:
z2 = append_membership_feature(X_train, y_train, X_test, fKernel = dot_kernel_norm, cluster = 2, L = 3, normalize=True)

Calculating kernel
Finding pseudo-optimal center coeffs
Minimization of J using COBYLA: function value at the initial point: 0.00037437607371591797
Done: function value at the final point: 2.4660617727302467e-05
Finding principal events
step: 0; selected: 4; sMax = 0.9978149983256913; progress = 0.9978149983256913
step: 1; selected: 3; sMax = 0.9980806035855869; progress = 0.0002656052598956071
step: 2; selected: 10; sMax = 0.9985688485789538; progress = 0.0004882449933668598
step: 3; selected: 20; sMax = 0.9985851424068907; progress = 1.6293827936886274e-05
Scale: 5.16040447145093e-06, membership range: 0.0008154457191053501 - 0.9991845542808947
Calculating memberships
Done


In [36]:
Mb = np.concatenate((np.append(z0['x_train'][:,[4]], z0['x_test'][:,[4]], axis = 0), 
               np.append(z1['x_train'][:,[4]], z1['x_test'][:,[4]], axis = 0),
               np.append(z2['x_train'][:,[4]], z2['x_test'][:,[4]], axis = 0)),
               axis = 1)
P = np.concatenate((np.append(z0['x_train'][:,-3:], z0['x_test'][:,-3:], axis = 0),
              np.append(z1['x_train'][:,-3:], z1['x_test'][:,-3:], axis = 0),
              np.append(z2['x_train'][:,-3:], z2['x_test'][:,-3:], axis = 0)),
              axis = 1)
C = np.append(y_train, y_test)


In [37]:
labels = ["%d: %.2f%%" % (C[j] , 100*Mb[j,0]) for j in range(len(C))]
plotly_scatter(data=P[:,0:3], Class = C, text = labels, marker_size = 3, fname = 'center-cetosa')

In [38]:
labels = ["%d: %.2f%%" % (C[j] , 100*Mb[j,1]) for j in range(len(C))]
plotly_scatter(data=P[:,3:6], Class = C, text = labels, marker_size = 3, fname = 'center-versicolor')

In [39]:
labels = ["%d: %.2f%%" % (C[j] , 100*Mb[j,2]) for j in range(len(C))]
plotly_scatter(data=P[:,6:9], Class = C, text = labels, marker_size = 3, fname = 'center-verginica')

In [40]:
N_train = X_train.shape[0]
#x_train = X_train
#x_test = X_test
x_train = np.append(X_train, Mb[0:N_train,[1]], axis = 1)
x_test = np.append(X_test, Mb[N_train:,[1]], axis = 1)

In [41]:
gs = model_selection.GridSearchCV(AdaBoostClassifier(random_state = my_seed),
                  param_grid={'n_estimators': range(5, N_train, 5), 'learning_rate': np.linspace(0.1, 2, num = 20) },
                  cv=5, n_jobs = 8)
gs.fit(x_train, y_train)


GridSearchCV(cv=5, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=777),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'n_estimators': range(5, 75, 5), 'learning_rate': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [42]:
gs.best_estimator_

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.5, n_estimators=5, random_state=777)

In [43]:
model = AdaBoostClassifier(n_estimators=gs.best_estimator_.n_estimators, random_state=my_seed, learning_rate=gs.best_estimator_.learning_rate)
model.fit(x_train, y_train)
Z = model.predict(x_train)
pd.crosstab(y_train, Z)

col_0,0,1,2
row_0,,,
0,22,0,0
1,0,30,2
2,0,0,21


In [44]:
Z_test = model.predict(x_test)
pd.crosstab(y_test, Z_test)

col_0,0,1,2
row_0,,,
0,28,0,0
1,0,17,1
2,0,1,28


In [45]:
model.feature_importances_

array([0. , 0. , 0.2, 0.2, 0.6])